In [10]:
import difflib

def compare_files(content1, content2):
    # 读取两个文件的内容
    file1_lines = content1.split('\n')
    file2_lines = content2.split('\n')

    ## 忽略空行
    file1_lines = [line for line in file1_lines if line.strip()]
    file2_lines = [line for line in file2_lines if line.strip()]

    # 使用difflib进行对比
    d = difflib.Differ()
    diff = d.compare(file1_lines, file2_lines)

    # 输出差异
    diff_result = '\n'.join(diff)
    return diff_result

# 使用示例
# file1 = '../pages/positive/constant-property-referencing-check-in-loops.ets'
# file2 = '../pages/negative/constant-property-referencing-check-in-loops.ets'
# result = compare_files(file1, file2)
# print(result)


In [2]:
import logging
from openai import OpenAI
import requests


def generate(query, system_prompt='You are a helpful AI assistant', base_url='https://xiaoai.plus/v1', model='gpt-4o-mini', retries=3):
    def call_ollama(query):
        data = {
            "model": model,
            "prompt": query,
            "stream": False
        }
        response = requests.post('http://localhost:11434/api/generate', json=data)
        return response.json()['response']
        
    def get_api_key(base_url):
        if base_url == 'https://xiaoai.plus/v1':
            # return "sk-Y7BDSlT4EKjhVfPpA603Bb0cC549424b9d1734262f6fE6C0"
            return "sk-2zahs8cnKDwpPkOxHdURa7uym5XCiLlHQ8RSfUPHnr5T075w"
        return "sk-bee3f43476e04c0ba5a0eee09961f325"
        
    def create_chat_completion(client, query, system_prompt=None):
        messages = query if isinstance(query, list) else (
            [{"role": "user", "content": query}] if 'o1-mini' in model
            else [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query}
            ]
        )
        return client.chat.completions.create(
            model=model,
            messages=messages
        )

    for attempt in range(retries):
        try:
            if model == 'arktsLLM':
                return call_ollama(query)
                
            client = OpenAI(
                base_url=base_url,
                api_key=get_api_key(base_url)
            )
            chat_completion = create_chat_completion(client, query, system_prompt)
            return chat_completion.choices[0].message.content
            
        except Exception as e:
            logging.getLogger().error(f"API调用第{attempt + 1}次失败: {e}")
            if attempt + 1 == retries:
                raise

In [11]:
import pandas as pd


xlsx_file = "./addition.xlsx"
df = pd.read_excel(xlsx_file)

for index, row in df.iterrows():
    file1 = row['问题代码样例']
    file2 = row['修复代码样例']
    # difflib = row['difflib']
    # gpt_diff = row['差异']
    # if gpt_diff is None:
    #     print(f"gpt_diff is None, {index}")
    
    result = compare_files(file1, file2)
    diff = generate(f"这个是arkts语言的问题代码样例：\n\n{file1}\n\n这个是修复代码样例：\n\n{file2}\n\n请对比这两个代码样例，并输出在修复过程中，修复代码样例相对于问题代码样例的差异")

    # 保存到excel
    df.loc[index, 'difflib'] = result
    df.loc[index, '差异'] = diff

df.to_excel(xlsx_file, index=False)

/tmp/ipykernel_3767840/110695034.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '  @Entry
  @Component
  struct MyComponent {
-   @State mOpacity: number = 1;
    @State show: boolean = true;
    build() {
      Column() {
        Row() {
          if (this.show) {
-           Text('value')
-             .opacity(this.mOpacity)
+           Text('value')// Set id to make transition interruptible
+             .id('myText')
+             .transition(TransitionEffect.OPACITY.animation({ duration: 1000 }))
          }
-       }
-       .width('100%')
+       }.width('100%')
?       +

        .height(100)
        .justifyContent(FlexAlign.Center)
        Text('toggle state')
          .onClick(() => {
+           // Through transition, animates the appearance or disappearance of transparency.
-           this.show = true;
?                        ^^^

+           this.show = !this.show;
?               

In [9]:
code1 = """

/*
 * Copyright (c) 2024 Shenzhen Kaihong Digital Industry Development Co., Ltd.
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *     http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

import { TitleBar } from '../../../../common/TitleBar';
import { LEFT_PADDING, RIGHT_PADDING } from '../../ComponentData';
import { WaterFlowScrollByControlPanel, WaterFlowScrollEdgeControlPanel, WaterFlowDataSource,
  WaterFlowPropertiesScrollBarControlPanel,
  WaterFlowPropertiesScrollBarColorControlPanel,
  WaterFlowPropertiesScrollBarWidthControlPanel,
  WaterFlowPropertiesEdgeEffectControlPanel,
} from './WaterFlowDataSource'

@Extend(Column)
function backgroundStyle() {
  .width('100%')
  .padding(12)
  .borderRadius(16)
  .backgroundColor(Color.White)
  .justifyContent(FlexAlign.SpaceAround)
}

@Extend(Text)
function listItemTextStyle() {
  .fontSize(15)
  .borderRadius(8)
  .textAlign(TextAlign.Center)
  .backgroundColor($r('app.color.background_grey'))
}
@Extend(Column) function columnStyle() {
  .width('100%')
  .padding(12)
  .borderRadius(16)
  .backgroundColor(Color.White)
  .justifyContent(FlexAlign.SpaceAround)
}

@Entry
@Component
struct WaterFlowSample {
  listDirection: Axis = Axis.Vertical;

  initIndex: number = AppStorage.get<number>('listSampleInitIndex') || 0;
  @State curScrollEvent: string = ""; // $r('app.string.current_event_value')
  @State scrollState: ScrollState = ScrollState.Idle
  @State scrollOffset: number = 10;
  @State firstIndex: number = 0;
  @State lastIndex: number = 0;
  currentIndex: number = 0;

  isChainAnimation: boolean = false;
  multiSelectable: boolean = false;
  lanes: number = 1;
  @State alignListItem: ListItemAlign = ListItemAlign.Center;

  @State @Watch("onScrollByChange") scrollBy: number = 100;
  @State @Watch("onChange") scrollEdge: Edge = Edge.Top;

  @State edgeEffect: EdgeEffect = EdgeEffect.Spring;
  @State scrollBarWidth: number = 0;
  @State scrollBarColor: Color = Color.Blue;
  @State scrollBar: BarState = BarState.On;

  private itemHeightArray: number[] = [];

  scroller: Scroller = new Scroller();
  dataSource: WaterFlowDataSource = new WaterFlowDataSource();

  onScrollByChange() {
    this.scroller.scrollBy(0, this.scrollBy);
  }

  onChange(){
    this.scroller.scrollEdge(this.scrollEdge);
  }

    // 保存flow item宽/高
    getItemSizeArray() {
      for (let i = 0; i < 100; i++) {
       if (i % 4 == 0) {
         this.itemHeightArray.push(102);
       } else if (i % 4 == 1) {
         this.itemHeightArray.push(136);
       } else if (i % 4 == 2) {
         this.itemHeightArray.push(136);
       }  else if (i % 4 == 3) {
         this.itemHeightArray.push(102);
       }
      }
    }

    aboutToAppear() {
      this.getItemSizeArray();
    }

  build() {
    Flex({ direction: FlexDirection.Column, alignItems: ItemAlign.Center, justifyContent: FlexAlign.Start }) {
      TitleBar({ title: $r('app.string.water_flow_title') })
        .flexGrow(0)

      Column() {
        WaterFlow({scroller: this.scroller}) {
          LazyForEach(this.dataSource, (item: number) => {
        FlowItem() {
          Column() {
            Image('resources/base/media/water_flow_'+ item % 4 + '.png')
              .borderRadius(16)
              .objectFit(ImageFit.Fill)
          }
          .height('100%')
          .borderRadius(16)
          .height(this.itemHeightArray[item % 4])
        }
        .width('100%')
        .backgroundColor(Color.White)
      }, (item: string) => item)
       .onAppear(() => {
         if (item + 10 == this.dataSource.totalCount()) {
           for (let i = 0; i < 50; i++) {
             this.dataSource.addLastItem()
           }
         }
       })
    }
        .columnsTemplate("1fr 1fr")
        .columnsGap(12)
        .rowsGap(12)
        .width('100%')
        .scrollBar(this.scrollBar)
        .scrollBarColor(this.scrollBarColor)
        .scrollBarWidth(this.scrollBarWidth)
        .edgeEffect(this.edgeEffect)
       .cachedCount(3)
      }
      .padding({ left: LEFT_PADDING, right: RIGHT_PADDING })
      .backgroundStyle()
      .height('45%')
      .width('100%')
      .flexGrow(0)

      // 状态栏
      Column() {
        Text() {
          Span($r('app.string.current_scroll_event'));
          Span(this.curScrollEvent);
        }

        Text() {
          Span($r('app.string.current_scroll_state'));
          Span(this.scrollState.toString());
        }

        Text() {
          Span($r('app.string.current_scroll_distance'));
          Span(this.scrollOffset.toString());
        }

        Text() {
          Span($r('app.string.current_scroll_index_first'));
          Span(this.firstIndex.toString());
          Span($r('app.string.current_scroll_index_last'));
          Span(this.lastIndex.toString());
        }
      }
      .height('16%')
      .width('100%')
      .padding({left:12})
      .alignItems(HorizontalAlign.Start)
      .justifyContent(FlexAlign.Center)
      .flexGrow(0)

      // tab 页签
        Column() {
          Tabs({ barPosition: BarPosition.Start }) {
            // 子页签：WaterFlow控制
            TabContent() {
              Column() {
                WaterFlowScrollByControlPanel({
                  scrollBy: $scrollBy
                })
                WaterFlowScrollEdgeControlPanel({
                  scrollEdge: $scrollEdge
                })
              }
              .width('100%')
              .height('100%')
            }.tabBar(new SubTabBarStyle($r('app.string.water_flow_controller')))

            // 子页签：WaterFlow属性
            TabContent() {
              Scroll() {
              Column() {
                WaterFlowPropertiesScrollBarControlPanel({scrollBar: $scrollBar})
                .backgroundColor(Color.White)
                .borderRadius(16)
                .padding({left:12,right:12,top:12})
                .margin({bottom:12})

                WaterFlowPropertiesScrollBarColorControlPanel({scrollBarColor: $scrollBarColor})
                .backgroundColor(Color.White)
                .borderRadius(16)
                .padding({left:12,right:12,top:12})
                .margin({bottom:12})

                WaterFlowPropertiesScrollBarWidthControlPanel({scrollBarWidth: $scrollBarWidth})
                  .backgroundColor(Color.White)
                  .borderRadius(16)
                  .padding({left:12,right:12,top:12})
                  .margin({bottom:12})

                WaterFlowPropertiesEdgeEffectControlPanel({edgeEffect: $edgeEffect})
                  .backgroundColor(Color.White)
                  .borderRadius(16)
                  .padding({left:12,right:12,top:12})
                  .margin({bottom:12})
              }
              .width('100%')
            }
            .flexGrow(1)
            .flexBasis(100)
            .margin({ bottom: 10 })
            .background()
            }.tabBar(new SubTabBarStyle($r('app.string.water_flow_properties')))

          }
          .vertical(false)
          .scrollable(true)
          .barMode(BarMode.Fixed)
          .onChange((index: number) => {
            console.info(index.toString())
          })
          .width('100%')
          .backgroundColor(0xF1F3F5)

        }
        .height('40%')
        .width('100%')
    }
    .padding({ left: LEFT_PADDING, right: RIGHT_PADDING })
    .height('100%')
    .width('100%')
    .backgroundColor($r('app.color.background_shallow_grey'))
  }
}
"""

code2 = """


/*
 * Copyright (c) 2024 Shenzhen Kaihong Digital Industry Development Co., Ltd.
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *     http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

import { TitleBar } from '../../../../common/TitleBar';
import { LEFT_PADDING, RIGHT_PADDING } from '../../ComponentData';
import { WaterFlowScrollByControlPanel, WaterFlowScrollEdgeControlPanel, WaterFlowDataSource,
  WaterFlowPropertiesScrollBarControlPanel,
  WaterFlowPropertiesScrollBarColorControlPanel,
  WaterFlowPropertiesScrollBarWidthControlPanel,
  WaterFlowPropertiesEdgeEffectControlPanel,
} from './WaterFlowDataSource'

@Extend(Column)
function backgroundStyle() {
  .width('100%')
  .padding(12)
  .borderRadius(16)
  .backgroundColor(Color.White)
  .justifyContent(FlexAlign.SpaceAround)
}

@Extend(Text)
function listItemTextStyle() {
  .fontSize(15)
  .borderRadius(8)
  .textAlign(TextAlign.Center)
  .backgroundColor($r('app.color.background_grey'))
}
@Extend(Column) function columnStyle() {
  .width('100%')
  .padding(12)
  .borderRadius(16)
  .backgroundColor(Color.White)
  .justifyContent(FlexAlign.SpaceAround)
}

@Entry
@Component
struct WaterFlowSample {
  listDirection: Axis = Axis.Vertical;

  initIndex: number = AppStorage.get<number>('listSampleInitIndex') || 0;
  @State curScrollEvent: string = ""; // $r('app.string.current_event_value')
  @State scrollState: ScrollState = ScrollState.Idle
  @State scrollOffset: number = 10;
  @State firstIndex: number = 0;
  @State lastIndex: number = 0;
  currentIndex: number = 0;

  isChainAnimation: boolean = false;
  multiSelectable: boolean = false;
  lanes: number = 1;
  @State alignListItem: ListItemAlign = ListItemAlign.Center;

  @State @Watch("onScrollByChange") scrollBy: number = 100;
  @State @Watch("onChange") scrollEdge: Edge = Edge.Top;

  @State edgeEffect: EdgeEffect = EdgeEffect.Spring;
  @State scrollBarWidth: number = 0;
  @State scrollBarColor: Color = Color.Blue;
  @State scrollBar: BarState = BarState.On;

  private itemHeightArray: number[] = [];

  scroller: Scroller = new Scroller();
  dataSource: WaterFlowDataSource = new WaterFlowDataSource();

  onScrollByChange() {
    this.scroller.scrollBy(0, this.scrollBy);
  }

  onChange(){
    this.scroller.scrollEdge(this.scrollEdge);
  }

    // 保存flow item宽/高
    getItemSizeArray() {
      for (let i = 0; i < 100; i++) {
       if (i % 4 == 0) {
         this.itemHeightArray.push(102);
       } else if (i % 4 == 1) {
         this.itemHeightArray.push(136);
       } else if (i % 4 == 2) {
         this.itemHeightArray.push(136);
       }  else if (i % 4 == 3) {
         this.itemHeightArray.push(102);
       }
      }
    }

    aboutToAppear() {
      this.getItemSizeArray();
    }

  build() {
    Flex({ direction: FlexDirection.Column, alignItems: ItemAlign.Center, justifyContent: FlexAlign.Start }) {
      TitleBar({ title: $r('app.string.water_flow_title') })
        .flexGrow(0)

      Column() {
        WaterFlow({scroller: this.scroller}) {
          LazyForEach(this.dataSource, (item: number) => {
        FlowItem() {
          Column() {
            Image('resources/base/media/water_flow_'+ item % 4 + '.png')
              .borderRadius(16)
              .objectFit(ImageFit.Fill)
          }
          .height('100%')
          .borderRadius(16)
          .height(this.itemHeightArray[item % 4])
        }
        .width('100%')
        .backgroundColor(Color.White)
      }, (item: string) => item)
    }
        .columnsTemplate("1fr 1fr")
        .columnsGap(12)
        .rowsGap(12)
        .width('100%')
        .scrollBar(this.scrollBar)
        .scrollBarColor(this.scrollBarColor)
        .scrollBarWidth(this.scrollBarWidth)
        .edgeEffect(this.edgeEffect)
       .cachedCount(3)
      }
      .padding({ left: LEFT_PADDING, right: RIGHT_PADDING })
      .backgroundStyle()
      .height('45%')
      .width('100%')
      .flexGrow(0)

      // 状态栏
      Column() {
        Text() {
          Span($r('app.string.current_scroll_event'));
          Span(this.curScrollEvent);
        }

        Text() {
          Span($r('app.string.current_scroll_state'));
          Span(this.scrollState.toString());
        }

        Text() {
          Span($r('app.string.current_scroll_distance'));
          Span(this.scrollOffset.toString());
        }

        Text() {
          Span($r('app.string.current_scroll_index_first'));
          Span(this.firstIndex.toString());
          Span($r('app.string.current_scroll_index_last'));
          Span(this.lastIndex.toString());
        }
      }
      .height('16%')
      .width('100%')
      .padding({left:12})
      .alignItems(HorizontalAlign.Start)
      .justifyContent(FlexAlign.Center)
      .flexGrow(0)

      // tab 页签
        Column() {
          Tabs({ barPosition: BarPosition.Start }) {
            // 子页签：WaterFlow控制
            TabContent() {
              Column() {
                WaterFlowScrollByControlPanel({
                  scrollBy: $scrollBy
                })
                WaterFlowScrollEdgeControlPanel({
                  scrollEdge: $scrollEdge
                })
              }
              .width('100%')
              .height('100%')
            }.tabBar(new SubTabBarStyle($r('app.string.water_flow_controller')))

            // 子页签：WaterFlow属性
            TabContent() {
              Scroll() {
              Column() {
                WaterFlowPropertiesScrollBarControlPanel({scrollBar: $scrollBar})
                .backgroundColor(Color.White)
                .borderRadius(16)
                .padding({left:12,right:12,top:12})
                .margin({bottom:12})

                WaterFlowPropertiesScrollBarColorControlPanel({scrollBarColor: $scrollBarColor})
                .backgroundColor(Color.White)
                .borderRadius(16)
                .padding({left:12,right:12,top:12})
                .margin({bottom:12})

                WaterFlowPropertiesScrollBarWidthControlPanel({scrollBarWidth: $scrollBarWidth})
                  .backgroundColor(Color.White)
                  .borderRadius(16)
                  .padding({left:12,right:12,top:12})
                  .margin({bottom:12})

                WaterFlowPropertiesEdgeEffectControlPanel({edgeEffect: $edgeEffect})
                  .backgroundColor(Color.White)
                  .borderRadius(16)
                  .padding({left:12,right:12,top:12})
                  .margin({bottom:12})
              }
              .width('100%')
            }
            .flexGrow(1)
            .flexBasis(100)
            .margin({ bottom: 10 })
            .background()
            }.tabBar(new SubTabBarStyle($r('app.string.water_flow_properties')))

          }
          .vertical(false)
          .scrollable(true)
          .barMode(BarMode.Fixed)
          .onChange((index: number) => {
            console.info(index.toString())
          })
          .width('100%')
          .backgroundColor(0xF1F3F5)

        }
        .height('40%')
        .width('100%')
    }
    .padding({ left: LEFT_PADDING, right: RIGHT_PADDING })
    .height('100%')
    .width('100%')
    .backgroundColor($r('app.color.background_shallow_grey'))
  }
}
"""
print(compare_files(code1, code2))

  /*
   * Copyright (c) 2024 Shenzhen Kaihong Digital Industry Development Co., Ltd.
   * Licensed under the Apache License, Version 2.0 (the "License");
   * you may not use this file except in compliance with the License.
   * You may obtain a copy of the License at
   *
   *     http://www.apache.org/licenses/LICENSE-2.0
   *
   * Unless required by applicable law or agreed to in writing, software
   * distributed under the License is distributed on an "AS IS" BASIS,
   * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   * See the License for the specific language governing permissions and
   * limitations under the License.
   */
  import { TitleBar } from '../../../../common/TitleBar';
  import { LEFT_PADDING, RIGHT_PADDING } from '../../ComponentData';
  import { WaterFlowScrollByControlPanel, WaterFlowScrollEdgeControlPanel, WaterFlowDataSource,
    WaterFlowPropertiesScrollBarControlPanel,
    WaterFlowPropertiesScrollBarColorControlPanel,
    WaterFlo